In [1]:
!python --version

Python 3.6.9 :: Anaconda, Inc.


In [2]:
# Installation https://nvidia.github.io/TRTorch/tutorials/installation.html#installation
!pip install  https://github.com/NVIDIA/TRTorch/releases/download/v0.0.2/trtorch-0.0.2-cp36-cp36m-linux_x86_64.whl


In [3]:
import torch 
from torch import nn
import torch.nn.functional as F

class LeNetFeatExtractor(nn.Module):
    def __init__(self):
        super(LeNetFeatExtractor, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)

    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        return x

class LeNetClassifier(nn.Module):
    def __init__(self):
        super(LeNetClassifier, self).__init__()
        self.fc1 = nn.Linear(16 * 6 * 6, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = torch.flatten(x,1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.feat = LeNetFeatExtractor()
        self.classifer = LeNetClassifier()

    def forward(self, x):
        x = self.feat(x)
        x = self.classifer(x)
        return x


In [4]:
model = LeNet()
model.eval()
traced_model = torch.jit.trace(model, torch.empty([1,1,32,32]))


In [5]:
traced_model

LeNet(
  original_name=LeNet
  (feat): LeNetFeatExtractor(
    original_name=LeNetFeatExtractor
    (conv1): Conv2d(original_name=Conv2d)
    (conv2): Conv2d(original_name=Conv2d)
  )
  (classifer): LeNetClassifier(
    original_name=LeNetClassifier
    (fc1): Linear(original_name=Linear)
    (fc2): Linear(original_name=Linear)
    (fc3): Linear(original_name=Linear)
  )
)

In [6]:

model = LeNet().eval()
script_model = torch.jit.script(model)


In [7]:
script_model

RecursiveScriptModule(
  original_name=LeNet
  (feat): RecursiveScriptModule(
    original_name=LeNetFeatExtractor
    (conv1): RecursiveScriptModule(original_name=Conv2d)
    (conv2): RecursiveScriptModule(original_name=Conv2d)
  )
  (classifer): RecursiveScriptModule(
    original_name=LeNetClassifier
    (fc1): RecursiveScriptModule(original_name=Linear)
    (fc2): RecursiveScriptModule(original_name=Linear)
    (fc3): RecursiveScriptModule(original_name=Linear)
  )
)

In [ ]:
import trtorch


compile_settings = {
    "input_shapes": [
        {
            "min" : [1, 1, 32, 32],
            "opt" : [1, 1, 33, 33],
            "max" : [1, 1, 34, 34],
        }
    ],
    "op_precision": torch.half # Run with FP16
}

#trt_ts_module = trtorch.compile(traced_model, compile_settings)
trt_ts_module = trtorch.compile(script_model, compile_settings)

input_data = torch.randn((1, 1, 32, 32))
input_data = input_data.half().to("cuda")


result = trt_ts_module(input_data)
torch.jit.save(trt_ts_module, "trt_ts_module.ts")

RuntimeError: Trying to create tensor with negative dimension -1: [1, 16, -1, -1]